In [1]:
import yfinance as yf
import vectorbt as vbt
import pandas as pd
import plotly.express as px
from datetime import datetime, timedelta

In [82]:
oversold = 25
period = 2
timeframe = "1d"

In [17]:
'''
The backtesting results were obtained using TradingView for the following parameters:
oversold <= 25
time period: 01-01-2015 to 30-05-2024
Only the best performing stocks were selected!
'''

backtesting_result = pd.read_csv("./ifr2_best_stocks_backtesting.csv")
backtesting_result

,Ticker,Total Trades,Profitable %,Gain Factor,Max Drawdown,Mean Transaction,Total Candles
0,CPLE6,237,71.31,2.521,38.86,1.09,5
1,TAEE11,220,76.35,2.576,14.87,0.82,5
2,VIVT3,248,77.42,2.769,13.94,0.99,4
3,ELET3,238,64.71,1.165,40.14,0.42,5
4,SLCE3,220,72.73,1.701,30.12,0.91,5
...,...,...,...,...,...,...,...
68,HYPE3,224,69.64,1.466,28.37,0.48,5
69,ALPA3,227,70.48,1.311,51.98,0.76,4
70,SBSP3,227,69.60,1.244,47.11,0.41,5
71,PETR3,218,68.35,1.356,66.05,0.82,5


In [117]:
# Define a custom scaling function
def custom_scale(series, scale_factor=10):
    min_val = series.min()
    max_val = series.max()
    return ((series - min_val) / (max_val - min_val) * scale_factor) + 1

# Apply the custom scaling function
backtesting_result['Total Candles Scaled'] = custom_scale(backtesting_result['Total Candles'])
backtesting_result["Greatness Factor"] = backtesting_result["Gain Factor"] * backtesting_result["Profitable %"]
backtesting_result["Greatness Factor Scaled"] = custom_scale(backtesting_result["Greatness Factor"], 50)
backtesting_result.sort_values(by="Greatness Factor", inplace=True)
backtesting_result.reset_index(inplace=True)

In [118]:
# Create 3D scatter plot
fig = px.scatter_3d(backtesting_result, x='Profitable %', y='Gain Factor', z='Total Trades', color="Max Drawdown", size="Total Candles Scaled", text="Ticker", size_max=30)

# Show plot
fig.show()

In [46]:
today = datetime.today()
i_date = (today - timedelta(days=30)).strftime('%Y-%m-%d')
f_date = today.strftime('%Y-%m-%d')
selected_stocks = [ticker + ".SA" for ticker in backtesting_result["Ticker"]]

# tickers = [ 
#     "PETR4.SA", "VALE3.SA", "ITUB4.SA", "BBDC4.SA", "MGLU3.SA", "ABEV3.SA", "BBAS3.SA", "B3SA3.SA",
#     "JBSS3.SA", "USIM5.SA", "WEGE3.SA", "SANB11.SA", "SUZB3.SA", "BRKM5.SA", "ELET3.SA",
#     "GGBR4.SA", "CEAB3.SA", "CSNA3.SA", "ITSA4.SA", "RDOR3.SA", "BEEF3.SA", "LREN3.SA",
#     "MULT3.SA", "HAPV3.SA", "EQTL3.SA", "TAEE11.SA", "EGIE3.SA", "KLBN11.SA", "TIMS3.SA", "BRFS3.SA",
#     "RAIL3.SA", "NTCO3.SA", "IGTI11.SA", "MRFG3.SA", "UGPA3.SA", "LIGT3.SA", "ARZZ3.SA", "CCRO3.SA",
#     "FLRY3.SA", "LWSA3.SA", "CYRE3.SA", "SOJA3.SA", "ENEV3.SA", "CPFE3.SA", "ALPA4.SA", "AESB3.SA",
#     "VIVA3.SA", "TOTS3.SA", "RADL3.SA", "RENT3.SA", "AALR3.SA", "IRBR3.SA", "GOLL4.SA",
#     "EZTC3.SA", "COGN3.SA", "POSI3.SA", "QUAL3.SA", "TEND3.SA", "JHSF3.SA",
#     "TUPY3.SA", "SAPR11.SA", "CMIG4.SA", "SIMH3.SA", "BRIT3.SA", "PETZ3.SA",
#     "BMOB3.SA", "BPAN4.SA", "ENJU3.SA", "MOVI3.SA", "TGMA3.SA", "BBSE3.SA", "MRVE3.SA",
#     "EMBR3.SA", "CSAN3.SA", "CIEL3.SA",
#     "NEOE3.SA", "ARZZ3.SA", "CVCB3.SA", "SBSP3.SA", "POMO4.SA", "PRIO3.SA",
#     "PSSA3.SA", "ECOR3.SA", "LOGG3.SA", "CXSE3.SA", "YDUQ3.SA", "PCAR3.SA", "AMER3.SA", 
#     "HYPE3.SA", "GOAU4.SA", "ENGI11.SA", "CRFB3.SA", "BRAP4.SA", "BPAC11.SA", "AZUL4.SA"
# ]

# Download data using yfinance
data = yf.download(selected_stocks, start=i_date, end=f_date, interval=timeframe)

[*********************100%%**********************]  73 of 73 completed


In [122]:
close = data["Adj Close"].iloc[-1]
var = (data['Close'].iloc[-1] - data['Close'].iloc[-2]) / data['Close'].iloc[-2] * 100

# Wilder method for RSI
# This calculation was made to match TradingView's RSI
def calculate_rsi(data, window):
    delta = data.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.ewm(alpha=1/window, min_periods=window).mean()
    avg_loss = loss.ewm(alpha=1/window, min_periods=window).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

rsi = calculate_rsi(data['Close'], period).iloc[-1]
# rsi = vbt.RSI.run(data["Adj Close"], window=2).rsi.iloc[-1].loc[(2)]

df = pd.DataFrame({
    'Close': close,
    'Var': var,
    'RSI': rsi,
}).reset_index(drop=False)

df["Ticker"] = df["Ticker"].str.replace(".SA", "") # Remove ".SA" suffix

In [135]:
# Filtering based on oversold condition
oversold_df = df[df["RSI"] <= oversold]
# Merging to get the previous calculated columns
oversold_df = oversold_df.merge(backtesting_result, how="left", on="Ticker") 
oversold_df.sort_values(by="Greatness Factor", ascending=False, inplace=True)
oversold_df.reset_index(inplace=True, drop=True)

In [133]:
# Create the scatter plot
fig = px.scatter(oversold_df, x='Greatness Factor', y='RSI', text='Ticker', color="Var", size_max=30)

# Update layout to improve appearance
fig.update_traces(textposition='top center')
fig.update_layout(title='RSI of Different Tickers',
                  yaxis= dict(title='RSI', range=[min(oversold_df['RSI']) - 5, max(oversold_df['RSI']) + 10], zeroline=True, zerolinewidth=2, zerolinecolor='LightGrey'),
                  xaxis=dict(title='Greatness Factor', range=[min(oversold_df["Greatness Factor"] - 40), max(oversold_df['Greatness Factor']) + 40], zeroline=True, zerolinewidth=2, zerolinecolor='LightGrey'),
                  height=600)

# Show plot
fig.show()

In [136]:
oversold_df

,Ticker,Close,Var,RSI,index,Total Trades,Profitable %,Gain Factor,Max Drawdown,Mean Transaction,Total Candles,Total Candles Scaled,Greatness Factor,Greatness Factor Scaled
0,TRPL3,34.500000,-1.597266,10.770215,53,193,83.94,3.439,18.58,1.69,4,1.0,288.66966,28.634555
1,CTKA4,17.510000,0.000000,7.281827,45,227,70.48,3.616,41.26,2.94,4,1.0,254.85568,24.287964
2,VIVT3,44.660000,-1.478054,3.376247,2,248,77.42,2.769,13.94,0.99,4,1.0,214.37598,19.084535
3,PTNT4,7.000000,-1.129942,18.619840,49,219,78.08,2.548,42.63,1.87,4,1.0,198.94784,17.101337
4,TAEE11,34.560001,-0.917430,9.065575,1,220,76.35,2.576,14.87,0.82,5,11.0,196.67760,16.809511
5,CPLE6,9.230000,-0.859300,7.516777,0,237,71.31,2.521,38.86,1.09,5,11.0,179.77251,14.636460
6,ENGI3,14.040000,-2.702705,21.067846,41,194,73.23,2.430,24.43,1.66,4,1.0,177.94890,14.402046
7,MNPR3,11.280000,-1.913046,0.467059,37,239,76.99,2.209,47.55,4.38,4,1.0,170.07091,13.389376
8,BPAC3,16.070000,-1.652390,17.938508,8,144,69.44,2.034,26.99,1.22,4,1.0,141.24096,9.683454
9,EQTL3,29.410000,-1.737389,14.109241,11,235,73.19,1.883,36.45,0.80,5,11.0,137.81677,9.243294
